In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv('bolig_data1.csv')

In [ ]:
#we start inspecting how our dataset looks like.
df.info()

In [ ]:
pd.set_option('display.max_rows',500)

In [ ]:
df.head()

In [ ]:
pcts = df.isnull().sum()/len(df)*100

In [ ]:
for null_col,pct in zip(df.columns[pcts>0],pcts[df.columns[pcts>0]]):
    print(f'{null_col}: {pct:.2f}% null')

In [ ]:
print(df.columns)

In [ ]:
#'monthly_rent' and 'Månedlig leje' both refer to "monthly rent" (one in English and one in Danish).
#'available_from' and 'Ledig fra' both refer to "available from" (one in English and one in Danish).
# 'Indflytningspris' refer to move_in_price which is the same as the english move_in_price column
# 'Lejeperiode' and rental_period both refer to rental period
# 'Aconto' and monthly_aconto both refer to aconto
# 'move_in_price' is a sum of other variables and we will not include it


# Drop the Danish versions if you want to keep the English ones
df.drop(['Månedlig leje', 'Ledig fra', 'Indflytningspris', 'Lejeperiode', 'Aconto','move_in_price'], axis=1, inplace=True)

In [ ]:
# Now for easier understanding of which columns we need for our project we will translate the columns from Danish to English
# Dictionary for translating column names
translations = {
    'breadcrumb': 'breadcrumb',
    'title': 'title',
    'description': 'description',
    'address': 'address',
    'monthly_rent': 'monthly_rent',
    'monthly_aconto': 'monthly_aconto',
    'move_in_price': 'move_in_price',
    'available_from': 'available_from',
    'rental_period': 'rental_period',
    'Boligtype': 'housing_type',  # Danish: Boligtype
    'Størrelse': 'size_sqm',  # Danish: Størrelse
    'Værelser': 'rooms',  # Danish: Værelser
    'Etage': 'floor',  # Danish: Etage
    'Møbleret': 'furnished',  # Danish: Møbleret
    'Delevenlig': 'roommate_friendly',  # Danish: Delevenlig
    'Husdyr tilladt': 'pets_allowed',  # Danish: Husdyr tilladt
    'Elevator': 'elevator',  # Danish: Elevator
    'Seniorvenlig': 'senior_friendly',  # Danish: Seniorvenlig
    'Kun for studerende': 'students_only',  # Danish: Kun for studerende
    'Altan/terrasse': 'balcony_terrace',  # Danish: Altan/terrasse
    'Parkering': 'parking',  # Danish: Parkering
    'Opvaskemaskine': 'dishwasher',  # Danish: Opvaskemaskine
    'Vaskemaskine': 'washing_machine',  # Danish: Vaskemaskine
    'Ladestander': 'charging_station',  # Danish: Ladestander
    'Tørretumbler': 'dryer',  # Danish: Tørretumbler
    'Lejeperiode': 'rental_period',  # Danish: Lejeperiode
    'Ledig fra': 'available_from',  # Danish: Ledig fra
    'Månedlig leje': 'monthly_rent',  # Danish: Månedlig leje
    'Aconto': 'aconto',  # Danish: Aconto
    'Depositum': 'deposit',  # Danish: Depositum
    'Forudbetalt husleje': 'prepaid_rent',  # Danish: Forudbetalt husleje
    'Indflytningspris': 'move_in_price',  # Danish: Indflytningspris
    'Oprettelsesdato': 'creation_date',  # Danish: Oprettelsesdato
    'Sagsnr.': 'case_number',  # Danish: Sagsnr.
    'energy_mark_src': 'energy_mark_source',
    'Energimærke ': 'energy_label'  # Danish: Energimærke
}

# Apply the translations to rename columns
df.rename(columns=translations, inplace=True)

In [ ]:
print(df.columns)

In [ ]:
#Check for Extra Spaces: It's possible that the column names have leading or trailing spaces, which is common when importing data.
# Let's clean the column names by stripping any unnecessary spaces.

# Strip any leading or trailing spaces from column names
df.columns = df.columns.str.strip()

In [ ]:
df.info()

In [ ]:
# We will now try to transform some of object data types to numeric ones. Mostly those that refer to prices.
columns_to_transform=['monthly_rent', 'monthly_aconto', 'deposit', 'prepaid_rent']
# Remove ' kr' and '.' for multiple columns
df[columns_to_transform] = df[columns_to_transform].apply(lambda x: x.str.replace('kr', '').str.replace('.', '').str.replace(',', '').str.strip() if x.str else '0')

In [ ]:
#df['move_in_price'] = df['move_in_price'].apply(lambda x: x.replace('måneder','').replace('Ubegrænset','0').replace('+','').split('-')[0])

In [ ]:
# column=['move_in_price']

# df[column]= df[column].apply(lambda x: x.str.replace(' kr', '').str.replace('.', '').str.replace(',', ''))

In [ ]:
df[columns_to_transform] = df[columns_to_transform].apply(pd.to_numeric)

In [ ]:
# In the move in price, whenever we have a value between 0 and 24 it's months of rent, so we multiply the months by monthly rent to get the accurate move_in_price and replace it with the number of months
# df.loc[(df['move_in_price']<25)&(df['move_in_price']>0),'move_in_price'] = df.loc[(df['move_in_price']<25)&(df['move_in_price']>0),'move_in_price']*df.loc[(df['move_in_price']<25)&(df['move_in_price']>0),'monthly_rent']

In [ ]:
# assumption: set prepaid rent to 0 when it's NaN

df['prepaid_rent'] = df['prepaid_rent'].fillna('0').astype(float)

In [ ]:
df[df.select_dtypes(include=['float']).columns] = df.select_dtypes(include=['float']).astype(int)

In [ ]:
df['energy_mark'] = df['energy_mark_source'].apply(lambda x: x.split('/')[-1].split('_')[0])

In [ ]:
df['size_sqm'] = df['size_sqm'].apply(lambda x: x.replace('m²','').strip().split('.')[0]).astype(int)

In [ ]:
df.drop(columns=['energy_mark_source','energy_label','breadcrumb','title','description','rental_period', 'case_number'], inplace=True)

In [ ]:
# Replacing As soon as possible in available_from column with the creation_date of the correspondent listing
df.loc[df['available_from'].str.contains('Snarest'),'available_from'] = df.loc[df['available_from'].str.contains('Snarest'),'creation_date']

In [ ]:
import datetime

# Dictionary to map Danish month names to numbers
danish_months = {
    " januar ": "1.",
    " februar ": "2.",
    " marts ": "3.",
    " april ": "4.",
    " maj ": "5.",
    " juni ": "6.",
    " juli ": "7.",
    " august ": "8.",
    " september ": "9.",
    " oktober ": "10.",
    " november ": "11.",
    " december ": "12."
}

def format_date(date_str):
    # Try to parse the date with the Danish month name
    for month, number in danish_months.items():
        if month in date_str:
            # Replace the month name with the corresponding number
            date_str = date_str.replace(month, number)
            # Parse the date to a datetime object
            #date_obj = datetime.datetime.strptime(date_str, "%d. %m. %Y")
            #return date_obj.strftime("%d.%m.%Y")
            return date_str
    # If it's already in the correct format
    #date_obj = datetime.datetime.strptime(date_str, "%d.%m.%Y")
    return date_str

In [ ]:
df['available_from'] = pd.to_datetime(df['available_from'].apply(format_date), dayfirst=True)

In [ ]:
df['creation_date'] = pd.to_datetime(df['creation_date'], dayfirst=True)

In [ ]:
df['area'] = df['address'].apply(lambda x: x.split('-')[0].split(',')[-1].strip() if '-' in x else x.split(',')[-1].strip())

In [ ]:
# We want to make floor a numeric var so we have to make assumptions: Stuen (=living room) is ground floor, Kælder (=cellar) is -1, - is translated to 0 as there is no floor
df['floor'] = df['floor'].apply(lambda x: x.replace('Stuen','0').replace('Kælder','-1').replace('-','0').replace('.','')).astype(int)

In [ ]:
df.drop(columns=['address'],inplace=True)

In [ ]:
for dtype, columns in df.columns.to_series().groupby(df.dtypes):
    print(f"Type: {dtype}")
    print(f"Columns: {list(columns)}\n")

In [ ]:
for col in df.columns:
    if df.dtypes[col] == 'O':
        print('###########################')
        print(col)
        print(df[col].unique(),end='\n\n')

In [ ]:
# create new column availability_in: buckets of <1 month, 1-3 months, 3+ months

df['availability_in'] = df.apply(lambda x: '<1 month' if (x['available_from']-x['creation_date']).days <30 else ('1-3 months' if (x['available_from']-x['creation_date']).days <90 else '3+ months'), axis = 1)

In [ ]:
scrape_date = pd.to_datetime('17-09-2024',dayfirst=True)
df['days_on_website'] = df['creation_date'].apply(lambda x: (scrape_date-x).days)

In [ ]:
df.drop(columns=['available_from','creation_date'],inplace=True)

In [ ]:
# defined as monthly_rent+aconto (in some cases aconto = 0 and we assume it is included in the rent, so to make the analysis more bulletproof
# we create a new variable total_monthly_rent to not drive misleading results)
df['total_monthly_rent'] = df['monthly_rent'] + df['monthly_aconto']

In [ ]:
"""
we create a new student_affordable column that, based on the total_monthly_rent,
it examines whether a student can afford renting the specific apartment or not
we were based on different reports on average student salary and other living costs
to estimate the threshold for an affordable option to 7500kr
"""
df['student_affordable'] = df['total_monthly_rent'] < 7500.00

In [ ]:
np.sum(df['student_affordable'])

In [ ]:
pd.set_option('display.max_columns',100)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
continuous_vars = ['monthly_rent','monthly_aconto','size_sqm','deposit','prepaid_rent','total_monthly_rent','days_on_website']
df[continuous_vars] = df[continuous_vars].astype(float)

In [ ]:
for i,type in enumerate(df.dtypes):
    print('- '+df.columns[i]+': '+str(type).replace('object','discrete/nominal').replace('int64','continuous/ordinal').replace('float64','continuous/ratio').replace('bool', 'discrete/nominal'))

In [ ]:
df.groupby(df['energy_mark']).count().iloc[:,0]/len(df)*100

In [ ]:
df['months_on_website'] = df['days_on_website'].apply(lambda x: '<1 month' if x<30 else ('1-3 months' if x<90 else ('3-6 months' if x <180 else '6+ months')))

In [ ]:
df.to_csv('preprocessed_data.csv', index=False, header=True, encoding='utf-8')